In [29]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

In [30]:
import torch

from llm_more_better.model import RewardModelLM
from transformers import AutoModelForCausalLM, AutoTokenizer

In [31]:
torch.set_printoptions(precision=3, linewidth=120, sci_mode=False)

In [32]:
model = RewardModelLM("meta-llama/Llama-3.2-1B-Instruct")

In [33]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")

In [13]:
tokenizer.pad_token = tokenizer.eos_token

In [19]:
prompt = [{"role" : "user", "content" : "Hello World"}]
tokens = tokenizer.apply_chat_template(prompt, return_tensors = "pt", padding = True)

In [21]:
tokens.shape

torch.Size([1, 33])

In [23]:
with torch.no_grad():
    with torch.autocast("cpu"):
        out = model(tokens)

In [4]:
model = RewardModelLM.load_from_checkpoint(
    "/teamspace/studios/this_studio/checkpoints/run_20241103_220303/best-checkpoint-epoch=04-val_loss=0.60.ckpt",
    dtype = torch.bfloat16
)

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


trainable params: 3,407,872 || all params: 1,239,222,272 || trainable%: 0.2750


RuntimeError: Error(s) in loading state_dict for RewardModelLM:
	Missing key(s) in state_dict: "model.score.layers.1.weight", "model.score.layers.4.weight". 
	Unexpected key(s) in state_dict: "model.score.weight". 

In [28]:
[p for p in model.parameters() if p.requires_grad]

[Parameter containing:
 tensor([[     0.003,      0.018,      0.021,  ...,     -0.005,     -0.042,     -0.033],
         [     0.024,     -0.023,      0.020,  ...,     -0.009,     -0.002,     -0.040],
         [     0.014,      0.011,      0.010,  ...,      0.007,     -0.012,      0.006],
         ...,
         [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010],
         [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010],
         [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010]], dtype=torch.bfloat16,
        requires_grad=True),
 Parameter containing:
 tensor([[-0.018,  0.007,  0.025,  ..., -0.009, -0.012,  0.020],
         [ 0.012,  0.059,  0.055,  ..., -0.033, -0.015,  0.011],
         [ 0.018,  0.016,  0.034,  ..., -0.039, -0.039, -0.028],
         ...,
         [ 0.030,  0.035,  0.071,  ..., -0.072, -0.026, -0.029],
         [ 0.023, -0.025,  0.035,  ..., -0.012, -0.029, -0.015],
         [-0.02

In [5]:
model.to(torch.bfloat16)

RewardModelLM(
  (model): PeftModelForCausalLM(
    (base_model): LoraModel(
      (model): LlamaForCausalLM(
        (model): LlamaModel(
          (embed_tokens): Embedding(128256, 2048)
          (layers): ModuleList(
            (0-15): 16 x LlamaDecoderLayer(
              (self_attn): LlamaSdpaAttention(
                (q_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=2048, out_features=16, bias=False)
                  )
                  (lora_B): ModuleDict(
                    (default): Linear(in_features=16, out_features=2048, bias=False)
                  )
                  (lora_embedding_A): ParameterDict()
                  (lora_embedding_B): ParameterDict()
                  (lora_magnitude_ve

In [6]:
lora_model = model.model

In [7]:
tokenizer = AutoTokenizer.from_pretrained(lora_model.config._name_or_path)

In [8]:
prompt = [{"role" : "user", "content" : "hello world", "role" : "assistant", "content" : "Hello. How can I assist you today?"}]
tokens = tokenizer.apply_chat_template(prompt, return_tensors = "pt")

In [9]:
prompt_advs = [{"role" : "user", "content" : "hello world", "role" : "assistant", "content" : "Fuck off"}]
tokens_advs = tokenizer.apply_chat_template(prompt_advs, return_tensors = "pt")

In [ ]:
model.eval()

with torch.no_grad():
    #with torch.autocast("cpu"):
    out = model(tokens)
    #out_advs = model(tokens_advs)

In [ ]:
out.sigmoid(), out_advs.sigmoid()

(tensor([[0.8281]], dtype=torch.bfloat16),
 tensor([[0.3438]], dtype=torch.bfloat16))

In [40]:
model.model.base_model.model.model.embed_tokens.weight

Parameter containing:
tensor([[     0.003,      0.018,      0.021,  ...,     -0.005,     -0.042,     -0.033],
        [     0.024,     -0.023,      0.020,  ...,     -0.009,     -0.002,     -0.040],
        [     0.014,      0.011,      0.010,  ...,      0.007,     -0.012,      0.006],
        ...,
        [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010],
        [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010],
        [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010]], dtype=torch.bfloat16)

In [41]:
model.model.base_model.model.lm_head.weight

Parameter containing:
tensor([[     0.003,      0.018,      0.021,  ...,     -0.005,     -0.042,     -0.033],
        [     0.024,     -0.023,      0.020,  ...,     -0.009,     -0.002,     -0.040],
        [     0.014,      0.011,      0.010,  ...,      0.007,     -0.012,      0.006],
        ...,
        [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010],
        [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010],
        [    -0.000,      0.011,     -0.019,  ...,      0.001,     -0.002,      0.010]], dtype=torch.bfloat16)

In [62]:
def test_generate(text):
    prompt = [{"role" : "user", "content" : text}]

    with torch.no_grad():
        with torch.autocast("cpu"):
            tokens = tokenizer.apply_chat_template(prompt, return_tensors = "pt")
            gen_out = model.model.generate(tokens, max_length = 1000)

    return tokenizer.decode(gen_out[0])

In [63]:
test_generate("hello world")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 08 Nov 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nhello world<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHello World!<|eot_id|>'

In [64]:
model.model.disable_adapter_layers()

In [65]:
test_generate("hello world")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 08 Nov 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nhello world<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHello World!<|eot_id|>'